In [ ]:
# Encode the CSV as utf-8 for python
import csv

path =  './training_set.csv'

with open(path, 'r', encoding='utf-8', errors='ignore') as infile, open('training_set_encoded.csv', 'w') as outfile:
     inputs = csv.reader(infile)
     output = csv.writer(outfile)

     for index, row in enumerate(inputs):
#          # Create file with no header
#          if index == 0:
#              continue
#          print(row)
         output.writerow(row)

In [16]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

In [17]:
# MacOS garbage
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [18]:
train_df = pd.read_csv('./training_set_encoded.csv')
train_df.head()

,Engagements,Followers at Posting,Created,Type,Description
0,502093,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...


In [19]:
train_sample_fraction = 0.3
y = train_df[['Engagements']]
# X = train_df.loc[:, train_df.columns != 'Engagements']
X = train_df[['Followers at Posting']]
# X.dtypes

In [25]:
rows = train_df.size
X['Year'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Month'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Day'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Hour'] = pd.DataFrame(0, index=range(rows), columns=range(1))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [26]:
dates = train_df[['Created']]

for r in range(dates.values.size):
    row = dates.values[r].tolist()
    row = "".join(row).split()
    date = row[0].split('-')
    year = date[0]
    month = date[1]
    day = date[2]
    hour = row[1].split(":")[0]
    X['Year'][r] = year
    X['Month'][r] = month
    X['Day'][r] = day
    X['Hour'][r] = hour

In [27]:
Media_Type = train_df[['Type']]

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Media_Type)
Media_Type = label_encoder.transform(Media_Type)
# print([x for x in Media_Type])
label_encoder.classes_

X['Type'] = Media_Type

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=train_sample_fraction, random_state=420)

# xgb_params = {
#     'objective': 'reg:linear',
#     'eval_metric': 'mae',
# }

xgb = XGBRegressor(
#     eta=0.3,
#     gamma=0.05,
#     max_depth=10,
#     min_child_weight=0.5,
#     max_delta_step=1,
#     reg_lambda=1,
#     alpha=1,
    objective='reg:linear',
    eval_metric='mae')

In [29]:
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

In [20]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [30]:
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print(MAPE)
MAE = mean_absolute_error(y_test, y_pred)
print(MAE)

38.05003877581727
57901.33581008584
